**Advaned Lane Finding Pipeline Outline**

- Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
- Apply a distortion correction to raw images.
- Use color transforms, gradients, etc., to create a thresholded binary image.
- Apply a perspective transform to rectify binary image ("birds-eye view").

- Detect lane pixels and fit to find the lane boundary.
- Determine the curvature of the lane and vehicle position with respect to center.

- Warp the detected lane boundaries back onto the original image.
- Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
import cv2
import glob, os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

%matplotlib inline

In [7]:

def plot_images(img1,img2,title1=None,title2=None,save_plot=False):
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(img1,cmap='gray')
    ax1.set_title(title1, fontsize=30)
    ax2.imshow(img2,cmap='gray')
    ax2.set_title(title2, fontsize=30)
    
    if save_plot:
        plot_fname = title1 + title2;
        f.savefig(plot_fname)

def show_perspective(img,warped_img,src,dst):
    # Plotting combined threshold images
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24,9))
    f.tight_layout()

    # Draw polygon to highlight the four source points on the image
    ax1.set_title('Thresholded Image with source points drawn',fontsize=30)
    ax1.plot(Polygon(src).get_xy()[:, 0], Polygon(src).get_xy()[:, 1], color='red',linewidth=4)
    ax1.imshow(img, cmap='gray') # display in grayscale
        
    ax2.set_title('Warped result with dest. points drawn',fontsize=30)
    # Draw polygon to highlight the four destination points on the image
    ax2.plot(Polygon(dst).get_xy()[:, 0], Polygon(dst).get_xy()[:, 1], color='red',linewidth=4)
    ax2.imshow(warped_img, cmap='gray')
    
    # Draw two vertical lines on the image
    ax2.axvline(x=270,color='red',linewidth=4)
    ax2.axvline(x=1040,color='red',linewidth=4)
    
    plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [8]:
def save_camera_calibration(ret,mtx,dist,rvecs,tvecs):
  
    # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
    dist_pickle = {}
    dist_pickle["ret"] = ret
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    dist_pickle["rvecs"] = rvecs
    dist_pickle["tvec"] = tvecs
    
    # Exclude the object and image points to save to the pickle file
    dist_pickle["objpoints"] = objpoints
    dist_pickle["imgpoints"] = imgpoints
            
    # Save the camera calibration pickle file to the "camera_cal" folder
    pickle.dump(dist_pickle, open("camera_cal/camera_cal.p", "wb" ))

def load_camera_calibration(camera_calibration='camera_cal/camera_cal.p'):
    if (os.path.isfile(camera_calibration)):
        dist_pickle = pickle.load( open(camera_calibration, "rb" ) )
        ret = dist_pickle["ret"]
        mtx  = dist_pickle["mtx"]
        dist = dist_pickle["dist"]
        rvecs = dist_pickle["rvecs"]
        tvecs = dist_pickle["tvec"]
    else:
        print('ERROR: Unable to open the camera calibration pickle file')
        return
    return ret, mtx, dist, rvecs, tvecs

def map_obj_img_points(img_name, objpoints, imgpoints, visible=True, h_corners=9, v_corners=6):
    # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((9*6,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    img = cv2.imread(img_name)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (h_corners,v_corners),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)
        
        if (visible==True):
            # Draw and display the corners
            img = cv2.drawChessboardCorners(img, (h_corners,v_corners), corners, ret)
            cv2.imshow('img',img)
            cv2.waitKey(500)
    else:
        # Notify the user that some images are not valid for camera calibration
        print("Unable to detect corners for image : " + img_name )
    return objpoints, imgpoints
    
def compute_camera_calibration(calibration_images):
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane
    
    # Make a list of calibration images
    images = glob.glob(calibration_images)
    
    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        objpoints, imgpoints = map_obj_img_points(fname,objpoints,imgpoints,visible=False)

    img_size = (img.shape[1], img.shape[0])
    
    # Do camera calibration given object points and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    return ret, mtx, dist, rvecs, tvecs

In [9]:
def undistort_camera_img(img,mtx,dist,show_distortion=False, save_undistort_image=False):   
    undist_img = cv2.undistort(img, mtx, dist, None, mtx)       
    # save undisort image
    if save_undistort_image:
        dist_name = os.path.basename(fname)
        undist_fname = 'output_images/'+ 'undist_' + os.path.splitext(dist_name)[0] + '.jpg'
        cv2.imwrite(undist_fname,undist_img)
    # show distortion comparison
    if show_distortion:
        plot_images(img,undist_img,title1='Camera Image',title2='Undisorted Camera Image')
    return undist_img

ret, mtx, dist, rvecs, tvecs = load_camera_calibration()
images = glob.glob('camera_cal/*.jpg')
for fname in images:
#     print(fname)
    # Read in an image
    img = mpimg.imread(fname)
    undistort_camera_img(img,mtx,dist,show_distortion=False, save_undistort_image=False)

ERROR: Unable to open the camera calibration pickle file


TypeError: cannot unpack non-iterable NoneType object